<a href="https://colab.research.google.com/github/buddypia/openai-agents-sdk-colab/blob/master/research_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## リサーチエージェント

検索キーワードに基づいて、複数エージェントが協力してリサーチを行う
- PlannerAgent
- WriterAgent
- SearchAgent

## INPUT
```markdown
ユーザーのキーワード
```

## OUTPUT
```markdown
キーワードに基づいてリサーチ結果
```

## エージェントフロー図
<img src="https://drive.google.com/uc?id=1BliW8WQEnHNkcEgfJlq9k7nSjTzxfAgF" alt="ColabにOpenAIのAPIキーの登録" width="80%">

In [11]:
!pip install openai-agents

In [12]:
import os
from google.colab import userdata

# ColabのシークレットからOpenAIのAPIキーを取得
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# OpenAIエージェントにAPIキーを設定
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [13]:
from pydantic import BaseModel

from agents import Agent

PROMPT = (
    "あなたは役に立つリサーチアシスタントです。クエリが与えられた場合、"
    "そのクエリに最もよく答えるために実行する一連のウェブ検索を考案してください。"
    "検索する用語を5から20個の間で出力してください。"
)

# WebSearchItemモデル：個々のウェブ検索アイテムを定義
class WebSearchItem(BaseModel):
    reason: str
    """この検索がクエリにとって重要である理由についてのあなたの推論。"""

    query: str
    """ウェブ検索に使用する検索語。"""


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """クエリに最もよく答えるために実行するウェブ検索のリスト。"""

# プランナーエージェントを定義
planner_agent = Agent(
    name="PlannerAgent",
    instructions=PROMPT,
    model="gpt-4o",
    output_type=WebSearchPlan,
)

In [14]:
from agents import Agent, WebSearchTool
from agents.model_settings import ModelSettings

INSTRUCTIONS = (
    "あなたはリサーチアシスタントです。検索語が与えられたら、その語でウェブを検索し、"
    "結果の簡潔な要約を作成します。要約は2〜3段落、300語未満でなければなりません。"
    "主要なポイントを捉えてください。簡潔に書き、完全な文や正しい文法は不要です。"
    "これはレポートを作成する人が利用するため、本質を捉え、冗長な部分を無視することが不可欠です。"
    "要約自体以外の追加のコメントは含めないでください。"
)

# サーチエージェントを定義
search_agent = Agent(
    name="SearchAgent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required"),
)

In [15]:
from pydantic import BaseModel

from agents import Agent

PROMPT = (
    "あなたは、ある調査クエリに対する一貫性のあるレポートを作成する任務を負った、シニアリサーチャーです。\n"
    "元のクエリと、リサーチアシスタントが行ったいくつかの初期調査が提供されます。\n"
    "まず、レポートの構造と流れを記述するレポートのアウトラインを作成する必要があります。\n"
    "次に、レポートを生成し、それを最終的な出力として返します。\n"
    "最終的な出力はマークダウン形式である必要があり、長く詳細なものにする必要があります。\n"
    "コンテンツは5〜10ページ、少なくとも1000語を目指してください。"
)

class ReportData(BaseModel):
    short_summary: str
    """調査結果の短い（2〜3文の）要約。"""

    markdown_report: str
    """最終レポート"""

    follow_up_questions: list[str]
    """さらなる調査のための推奨トピック"""

# ライターエージェントを定義
writer_agent = Agent(
    name="WriterAgent",
    instructions=PROMPT,
    model="o3-mini",
    output_type=ReportData,
)

In [16]:
import asyncio
import time

from agents import Runner, custom_span, gen_trace_id, trace

class ResearchManager:
  async def run(self, query: str) -> None:
      # 検索計画を立てる
      search_plan = await self._plan_searches(query)
      # 検索を実行する
      search_results = await self._perform_searches(search_plan)
      # レポートを作成する
      report = await self._write_report(query, search_results)

      # 最終レポート（短い要約）
      final_report = f"レポート要約\n\n{report.short_summary}"

      print("\n\n=====レポート=====\n\n")
      print(f"レポート: {report.markdown_report}")
      print("\n\n=====追加質問=====\n\n")
      follow_up_questions = "\n".join(report.follow_up_questions)
      print(f"追加質問: {follow_up_questions}")

  async def _plan_searches(self, query: str) -> WebSearchPlan:
    # プランナーエージェントを実行して検索計画を取得
    result = await Runner.run(
      planner_agent,
      f"クエリ: {query}", # ユーザーの元のクエリ
    )

    # 結果をWebSearchPlanオブジェクトとして返す
    return result.final_output_as(WebSearchPlan)

  async def _perform_searches(self, search_plan: WebSearchPlan) -> list[str]:
      # "ウェブ検索" という名前でカスタムスパンを開始
      with custom_span("ウェブ検索"):
          num_completed = 0 # 完了した検索タスクの数
          # 検索計画内の各項目に対して非同期検索タスクを作成
          tasks = [asyncio.create_task(self._search(item)) for item in search_plan.searches]
          results = [] # 検索結果を格納するリスト
          # 完了したタスクから順に結果を処理
          for task in asyncio.as_completed(tasks):
              result = await task # タスクの結果を取得
              if result is not None: # 結果がNoneでない場合のみ追加
                  results.append(result)
              num_completed += 1 # 完了数をインクリメント
          return results # 検索結果のリストを返す

  async def _search(self, item: WebSearchItem) -> str | None:
      # 検索エージェントへの入力を作成
      input = f"検索語: {item.query}\n検索理由: {item.reason}"
      try:
          # 検索エージェントを実行
          result = await Runner.run(
              search_agent,
              input,
          )
          # 結果を文字列として返す
          return str(result.final_output)
      except Exception:
          # エラーが発生した場合はNoneを返す
          return None

  async def _write_report(self, query: str, search_results: list[str]) -> ReportData:
      # ライターエージェントへの入力を作成
      input = f"元のクエリ: {query}\n要約された検索結果: {search_results}"
      # ライターエージェントをストリームモードで実行
      result = await Runner.run(
          writer_agent,
          input,
      )

      # 最終的な結果をReportDataオブジェクトとして返す
      return result.final_output_as(ReportData)

In [17]:
query = "サイバーエージェントのAI事業本部について"

In [18]:
await ResearchManager().run(query)

ERROR:openai.agents:No active trace. Make sure to start a trace with `trace()` firstReturning NoOpSpan.




=====レポート=====


レポート: # サイバーエージェントAI事業本部における戦略、技術、倫理指針、及びイノベーションの包括的考察

本レポートは、サイバーエージェントのAI事業本部の全体像を明らかにするとともに、同社がどのように先端技術や倫理基準、内部/外部のネットワーク強化を通じて企業戦略を推進しているかを検証します。以下に、レポートの構成と各セクションで扱う内容を示します。

## アウトライン

1. **はじめに**
   - レポートの目的と重要性
   - サイバーエージェントのAI事業本部の概要

2. **サイバーエージェントAI事業本部の背景と設立経緯**
   - 設立の背景（2019年9月設立の意義）
   - 広告事業および新規AI事業の統合

3. **主要技術研究領域と組織体制**
   - AI Labの研究範囲（機械学習、自然言語処理、コンピュータビジョン等）
   - Data Science Centerの役割と成果
   - MLOpsおよび生成AIの活用事例

4. **プロジェクトと実践活動**
   - 広告クリエイティブの自動生成や効果予測
   - ゲームAI Lab、アニメーションAI Labの取り組み
   - 医療や小売部門でのAI活用事例
   - コラボレーション事例（ベネッセホールディングス、シンギュレイト等）

5. **倫理指針と社会的影響への配慮**
   - 研究倫理ガイドラインの制定とその意義
   - 内部ガイドライン（画像生成AI、生成AI利用）の策定
   - 社会や環境への配慮とリスクマネジメント

6. **社内イベント・ネットワーク・キャリアパス**
   - 社内外のイベント・カンファレンスの開催
   - 人材育成、キャリアパス、異動制度
   - エンジニアMeetupや技術カンファレンスの取り組み

7. **今後の展望と課題**
   - 技術革新の方向性と市場環境
   - 組織間連携の強化と新たなフィールドへの進出
   - 倫理とプライバシー保護のさらなる推進

8. **結論**
   - 全体のまとめ
   - サイバーエージェントAI事業本部の競争力と未来展望

---

## 本文

### 1. はじめに

サイバーエージェントは、デジタル広